### Solar System Object Colors ###

This functions as a demo notebook of how to calculate colors of objects (in general),
but is specifically targeted and extended toward solar system objects. 

Solar system objects usually have reported reflectance spectra, rather than the
'as-observed' spectra. 

This notebook calculates synthetic colors for a sample of SMASSII reflectance spectra, and compares the results in LSST colors compared to SDSS colors. The results suggest that classification based on SDSS colors should work well in LSST, although the g band filter is likely not quite as optimal as SDSS (pending delivered filter curves). For objects bright enough to be visible in y band, the results may be even better as we will be able to prove new wavelength ranges. 

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import colorcet
import cycler
import pandas as pd
from scipy import signal
from scipy.interpolate import interp1d
import rubin_sim.phot_utils as phot_utils
from rubin_sim.data import get_data_dir

For a starting point, let's demonstrate going from a reflectance spectra to an observed SED, and then observed colors (in LSST and/or other systems) using rubin_sim. 

In [ ]:
# We have some sample asteroid spectra in rubin_sim_data_dir - this is what 'get_data_dir' returns.
# However, those are 'observed' spectra (already includes a sample solar spectra)
ast_templates = [k.replace('.dat.gz', '') for k in os.listdir(os.path.join(get_data_dir(), 'movingObjects')) 
                 if 'kurucz' not in k and 'harris' not in k]

In [ ]:
# Read the sso seds from disk (using rubin_sim.photUtils.Sed)
sso = {}
for k in ast_templates:
    sso[k]= phot_utils.Sed()
    sso[k].read_sed_flambda(os.path.join(get_data_dir(), 'movingObjects', f'{k}.dat.gz'))

# Read the sun too
sun = phot_utils.Sed()
sun.read_sed_flambda(os.path.join(get_data_dir(), 'movingObjects', 'kurucz_sun.gz'))

In [ ]:
# Let's visualize, with maybe a better way of cycling through the colors for the linestyles

fig = plt.figure(figsize=(8, 5))
ax = plt.gca()

# Pick a subset to plot to make it less confusing
subset = ['TNO', 'D', 'C', 'S', 'X', 'V']
color = cm.rainbow(np.linspace(0, 1, len(subset)))
ax.set_prop_cycle(cycler.cycler('color', color))

# Normalize the SEDs so they fit on the same plot but still make a bit more sense
# This particular normalization essentially makes all of the objects (and the sun) have
# the same total flux over the wwavelen_min to wwavelen_max window
wwavelen_min = 200
wwavelen_max = 2000
sun_window = np.where((sun.wavelen > wwavelen_min) & (sun.wavelen < wwavelen_max))
sun_max = sun.flambda[sun_window].mean()

for k in subset:
    sso_window = np.where((sso[k].wavelen > wwavelen_min) & (sso[k].wavelen < wwavelen_max))
    norm = sun_max / (sso[k].flambda[sso_window].mean())
    plt.plot(sso[k].wavelen, sso[k].flambda * norm, label=k, linestyle=':')
plt.plot(sun.wavelen, sun.flambda, 'k',  linestyle='-', label='Kurucz model sun')
plt.legend(loc=(1.01, 0.0))
plt.xlim(200, 2000)
plt.xlabel('Wavelength (nm)', fontsize='large')
plt.ylabel('F_lambda  (ergs/cm^s/s/nm)', fontsize='large')

With these 'observed' spectra, it's pretty straight forward to calculate expected magnitudes in the LSST bandpasses.

In [ ]:
# Read in the lsst total system bandpasses (this is QE, filters, telescope and sample atmosphere)
lsst = {}
lsst_filterlist = 'ugrizy'
for f in lsst_filterlist:
    lsst[f] = phot_utils.Bandpass()
    lsst[f].read_throughput(os.path.join(get_data_dir(), 'throughputs', 'baseline', f'total_{f}.dat'))

cmap = colorcet.cm['CET_R1'](np.linspace(0, 1, len(lsst_filterlist)))

fig = plt.figure(figsize=(8,5))
for i, f in enumerate(lsst_filterlist):
    plt.plot(lsst[f].wavelen, lsst[f].sb, color=cmap[i])
plt.xlabel('Wavelength (nm)', fontsize='large')
plt.ylabel('Sb (throughput, 0-1)', fontsize='large')
plt.grid(alpha=0.3)
plt.ylim(0, 1)
plt.axvline(450)
plt.axvline(925)

In [ ]:
# Note that we can also read in other bandpasses from the throughputs directory -- 
os.listdir(os.path.join(get_data_dir(), 'throughputs'))

In [ ]:
# Read in the sdss total system bandpasses (this is QE, filters, telescope and sample atmosphere, again)
sdss = {}
sdss_filterlist = 'ugriz'
for f in sdss_filterlist:
    sdss[f] = phot_utils.Bandpass()
    sdss[f].read_throughput(os.path.join(get_data_dir(), 'throughputs', 'sdss', f'sdss_{f}.dat'))
    sdss[f].resample_bandpass()

cmap = colorcet.cm['CET_R1'](np.linspace(0, 1, len(lsst_filterlist)))

fig = plt.figure(figsize=(8,5))
for i, f in enumerate(sdss_filterlist):
    plt.plot(sdss[f].wavelen, sdss[f].sb, color=cmap[i])
for i, f in enumerate(lsst_filterlist):
    plt.plot(lsst[f].wavelen, lsst[f].sb, linestyle=':', color=cmap[i])
plt.xlabel('Wavelength (nm)', fontsize='large')
plt.ylabel('Sb (throughput, 0-1)', fontsize='large')
plt.grid(alpha=0.3)
plt.ylim(0, 1)

Calculating magnitudes in each of these bandpasses. 

Note that really we're looking for colors, so the overall normalization of the fluxes for each object (or the magnitudes in any given band) are not important; just the relative differences of the magnitudes. We'll use 'lsst r' as the reference for now. 

In [ ]:
refband_name = 'r'
refband = lsst[refband_name]

# Calculate the reference bandpass magnitudes
refmags = {}
for k in sso:
    refmags[k] = sso[k].calc_mag(refband)

# Calculate the colors
colors = {}
for k in sso:
    colors[k] = {}
    for f in lsst_filterlist:
        colors[k][f'lsst_{f}'] = sso[k].calc_mag(lsst[f]) - refmags[k]
        
        
# Let's calculate colors for SDSS -- but let's update refband so we're using sdss r 
refband = sdss[refband_name]

# Calculate the reference bandpass magnitudes
refmags = {}
for k in sso:
    refmags[k] = sso[k].calc_mag(refband)

# Calculate the colors
for k in sso:
    for f in sdss_filterlist:
        colors[k][f'sdss_{f}'] = sso[k].calc_mag(sdss[f]) - refmags[k]

In [ ]:
df_colors = pd.DataFrame(colors)
df_colors

In [ ]:
# Also some people like V-x measurements.. 
Vband = phot_utils.Bandpass()
Vband.read_throughput(os.path.join(get_data_dir(), 'movingObjects', 'harris_V.dat'))

refband = Vband
# Calculate the reference bandpass magnitudes
refmags = {}
for k in sso:
    refmags[k] = sso[k].calc_mag(refband)

Vcolors = {}
# Calculate the colors
for k in sso:
    Vcolors[k] = {}
    for f in lsst_filterlist:
        Vcolors[k][f'V-{f}'] = refmags[k] - sso[k].calc_mag(lsst[f])
pd.DataFrame(Vcolors).T

In [ ]:
# Plot some color-color combinations
c1 = 'lsst_g'  #g-r
cc1 = 'sdss_g'
c2 = 'lsst_i'  #i-r .. we'll flip so it's r-i
cc2 = 'sdss_i'

fig = plt.figure(figsize=(8,8)) 
cmap = colorcet.cm['CET_R2'](np.linspace(0, 1, len(sso)))

sso_keys = sorted(list(sso.keys()))
for i, k in enumerate(sso_keys):
    c = cmap[i]
    plt.plot(-colors[k][c2], colors[k][c1], linestyle='', marker='o', color=c, label=k)
    plt.plot(-colors[k][cc2], colors[k][cc1], linestyle='', marker='x', color=c)
    plt.plot([-colors[k][c2], -colors[k][cc2]],
            [colors[k][c1], colors[k][cc1]], linestyle='-', marker='', color=c)
plt.xlabel(f"{refband_name} - {c2.replace('lsst_', '')}")
plt.ylabel(f"{c1.replace('lsst_', '')} - {refband_name}")
plt.legend(loc=(1.01, 0.0))

In [ ]:
# Plot some color-color combinations
c1 = 'lsst_g'  #g-r
cc1 = 'sdss_g'
c2 = 'lsst_z'  #z-r .. we'll flip so it's r-z
cc2 = 'sdss_z'

fig = plt.figure(figsize=(8,8)) 
cmap = colorcet.cm['CET_R2'](np.linspace(0, 1, len(sso)))

sso_keys = sorted(list(sso.keys()))
for i, k in enumerate(sso_keys):
    c = cmap[i]
    plt.plot(-colors[k][c2], colors[k][c1], linestyle='', marker='o', color=c, label=k)
    plt.plot(-colors[k][cc2], colors[k][cc1], linestyle='', marker='x', color=c)
    plt.plot([-colors[k][c2], -colors[k][cc2]],
            [colors[k][c1], colors[k][cc1]], linestyle='-', marker='', color=c)
plt.xlabel(f"{refband_name} - {c2.replace('lsst_', '')}")
plt.ylabel(f"{c1.replace('lsst_', '')} - {refband_name}")
plt.legend(loc=(1.01, 0.0))

So that's interesting and I'm not sure I quite understand what's going on. It's likely understandable that $z$ band colors change, as the $z$ bands in LSST are fairly different from SDSS. But this seems to show a big change in g-r colors as well, which I wasn't expecting. Could it be due to the slightly wider gap between g and r in SDSS? (this seems possible; it's a wavelength range where the asteroid observed SEDs are changing strongly due to the slope of the Sun's SED). 

Anyway .. let's see if we get any further insights after loading up a range of reflectance spectra (which we can also demonstrate turning into the SEDs we use to calculate photometry).

I downloaded the set of reflectance spectra from SMASS2 (Bus & Binzel), let's see what we can get.
http://smass.mit.edu/smass.html

https://sbn.psi.edu/pds/resource/smass2.html

citation: "Bus, S. and Binzel, R. P. (2020) Small Main-belt Asteroid Spectroscopic Survey, Phase II V1.0. urn:nasa:pds:gbo.ast.smass2.spectra::1.0. NASA Planetary Data System; https://doi.org/10.26033/fj1d-vb37." 

In [ ]:
# grab from https://sbnarchive.psi.edu/pds4/non_mission/gbo.ast.smass2.spectra.zip
ddir = 'gbo.ast.smass2.spectra/data'

obj_files = glob.glob(f'{ddir}/data*/*.tab')
test_file = obj_files[8]
print(len(obj_files), test_file)

In [ ]:
##### The SMASS2 data covers 435 - 925 nm. This means r, i, z band are relatively well covered,
# but only ~half of g band is inside this wavelength range. We'll extrapolate at least to cover g.
# I also extrapolated over u and y band, but these are probably really unreliable.

def extend_smass(filename):
    tt = pd.read_table(test_file, delim_whitespace=True, names=['wavelen', 'reflectance', 'reflectance_error'])
    # A few files have extraneous wavelength = 0 entries so let's remove those
    tt.query('wavelen > 0', inplace=True)
    tt.wavelen = tt.wavelen * 1000 # convert to nm
    # Get a smoothed version
    # The spectra are typically constant wavelength spacing within some range, then a jump, then constant spacing
    # Seems like this occurs around ~515 and ~750 nm (instrument configuration changes?)
    # window length * 2.5nm = wavelength range (35 ~ 87nm )
    sf = signal.savgol_filter(tt.reflectance,
                                    window_length=35, 
                                    polyorder=3)

    sf2 = signal.savgol_filter(tt.reflectance,
                                window_length=35, 
                                polyorder=1)

    tt['smoothed_reflectance'] = np.where(tt.reflectance_error > 0.03 * tt.reflectance, sf2, sf)
    
    # Note that all of these SEDs only reaches from 425 to 925 nm, cutting off at the red end of z band 
    # and midway through the g band, also missing all of the u and y bands (so those colors will not be reliable)

    # Fit the slope for the blue end and try to extend it 
    bluest = tt.wavelen.min()
    condition = ((tt.wavelen>bluest) & (tt.wavelen<bluest+100))    
    wavelen_extend = np.arange(300, bluest+0.5, 1)
    interp = interp1d(tt.wavelen[condition], tt.smoothed_reflectance[condition], 
                      kind='slinear', fill_value='extrapolate')
    reflect_extend = interp(wavelen_extend)

    wavelen_final = np.concatenate((wavelen_extend, tt.wavelen.values))
    reflect_final = np.concatenate((reflect_extend, tt.smoothed_reflectance.values))
    
    # After some attempts to extend the red end .. I think this needs something 
    # better than a simple extrapolation, due to various actual bumps in the spectra that 
    # happen near 900 nm. But we can give it a go and remember y band is probably wrong.
    condition = ((tt.wavelen>850) & (tt.wavelen < 925))
    wavelen_extend = np.arange(tt.wavelen.max(), 1103, 1)
    interp = interp1d(tt.wavelen[condition], tt.smoothed_reflectance[condition],
                      kind='slinear', fill_value='extrapolate')
    reflect_extend = interp(wavelen_extend)
    wavelen_final = np.concatenate([wavelen_final, wavelen_extend])
    reflect_final=np.concatenate([reflect_final, reflect_extend])
    
    
    ast_sed = phot_utils.Sed()
    ast_sed.set_sed(wavelen=wavelen_final, flambda=reflect_final)

    return ast_sed

In [ ]:
# Read the files and add the extrapolations
smass = {}
names = []
for test_file in obj_files: #['a000391.spfit', 'a005379.spfit', 'a002579.spfit']:
    name = os.path.split(test_file)[-1].split('_')[0]
    if name in names:
        print(name)
    names.append(name)
    smass[name] = extend_smass(test_file)

In [ ]:
plt.figure(figsize=(8, 5))
for k in smass:
    plt.plot(smass[k].wavelen, smass[k].flambda)
plt.axvline(925)
plt.axvline(825, color='r', linestyle=':')
plt.axvline(450)
plt.axvline(525, color='r', linestyle=':')
plt.title('Wavelength (nm)')
plt.ylabel('Reflectance')
plt.title('SMASS II')

Well -- anyway, that's probably close enough. 
Let's go ahead and turn them into SEDs we can use for calculating observed magnitudes. 

In [ ]:
# Limit the wavelength range for the sun, just to reduce warning messages
sun_limited = phot_utils.Sed()
idx = np.where((sun.wavelen > 299) & (sun.wavelen < 1105))
sun_limited.set_sed(wavelen=sun.wavelen[idx], flambda=sun.flambda[idx])
sun_limited.resample_sed(wavelen_match=lsst['g'].wavelen)
wavelen_step = np.unique(np.diff(lsst['g'].wavelen)).min()
obs_smass = {}
for k in smass:
    smass[k].resample_sed(wavelen_match=lsst['g'].wavelen)
    obs_smass[k] = smass[k].multiply_sed(sun_limited, wavelen_step=wavelen_step)

In [ ]:
plt.figure(figsize=(8, 5))
for k in smass:
    plt.plot(obs_smass[k].wavelen, obs_smass[k].flambda)
plt.xlabel('Wavelength (nm)')
plt.ylabel('Flambda')

In [ ]:
# Okay - so note that the filters have a wavelength grid that extends beyond the region that is reliable for the 
# input SEDS. The easiest way to handle this is to cut off the filters at long wavelengths.
# Note, if you get Nans at any point in your magnitudes, it's because your filter is including a range of wavelength
# that is not covered by the SED.

lsst = {}
lsst_filterlist = 'ugrizy'
for f in lsst_filterlist:
    lsst[f] = phot_utils.Bandpass()
    lsst[f].read_throughput(os.path.join(get_data_dir(), 'throughputs', 'baseline', f'total_{f}.dat'))
    lsst[f].resample_bandpass(wavelen_min=300, wavelen_max=1000)
sdss = {}
sdss_filterlist = 'ugriz'
for f in sdss_filterlist:
    sdss[f] = phot_utils.Bandpass()
    sdss[f].read_throughput(os.path.join(get_data_dir(), 'throughputs', 'sdss', f'sdss_{f}.dat'))
    sdss[f].resample_bandpass(wavelen_min=300, wavelen_max=1000)

In [ ]:
# And now we can calculate colors again - 
refband_name = 'r'
refband = lsst[refband_name]

# Calculate the reference bandpass magnitudes
refmags = {}
for k in smass:
    refmags[k] = smass[k].calc_mag(refband)

# Calculate the colors
colors = {}
for k in smass:
    colors[k] = {}
    for f in 'ugriz':
        # Don't include y because the SEDs are not good at these wavelengths.
        colors[k][f'lsst_{f}'] = smass[k].calc_mag(lsst[f]) - refmags[k]
        
        
# Let's calculate colors for SDSS -- but let's update refband so we're using sdss r 
refband = sdss[refband_name]

# Calculate the reference bandpass magnitudes
refmags = {}
for k in smass:
    refmags[k] = smass[k].calc_mag(refband)

# Calculate the colors
for k in smass:
    for f in sdss_filterlist:
        colors[k][f'sdss_{f}'] = smass[k].calc_mag(sdss[f]) - refmags[k]

df = pd.DataFrame(colors)
df

In [ ]:
# Plot some color-color combinations
c1 = 'lsst_g'  #g-r
cc1 = 'sdss_g'
c2 = 'lsst_i'  #i-r .. we'll flip so it's r-i
cc2 = 'sdss_i'

fig = plt.figure(figsize=(8,8)) 

smass_keys = sorted(list(smass.keys()))
for i, k in enumerate(smass_keys):
    plt.plot(-colors[k][c2], colors[k][c1], linestyle='', marker='.', color='r', label=k)
plt.xlabel(f"{refband_name} - {c2}")
plt.ylabel(f"{cc1} - {refband_name}")
#plt.legend(loc=(1.01, 0.0))

fig = plt.figure(figsize=(8,8)) 
for i, k in enumerate(smass_keys):
    plt.plot(-colors[k][cc2], colors[k][cc1], linestyle='', marker='.', color='g')
plt.xlabel(f"{refband_name} - {c2}")
plt.ylabel(f"{cc1} - {refband_name}")

In [ ]:
# Plot some color-color combinations
c1 = 'lsst_g'  #g-r
cc1 = 'sdss_g'
c2 = 'lsst_z'  #z-r .. we'll flip so it's r-z
cc2 = 'sdss_z'

fig = plt.figure(figsize=(8,8)) 

smass_keys = sorted(list(smass.keys()))
for i, k in enumerate(smass_keys):
    plt.plot(-colors[k][c2], colors[k][c1], linestyle='', marker='.', color='r', label=k)
plt.xlabel(f"{refband_name} - {c2}")
plt.ylabel(f"{c1} - {refband_name}")
#plt.legend(loc=(1.01, 0.0))

fig = plt.figure(figsize=(8,8)) 
for i, k in enumerate(smass_keys):
    plt.plot(-colors[k][cc2], colors[k][cc1], linestyle='', marker='.', color='g')
plt.xlabel(f"{refband_name} - {cc2}")
plt.ylabel(f"{cc1} - {refband_name}")

In [ ]:
plt.figure(figsize=(8,5))
for f in 'giz':
    plt.hist(df.loc[f'lsst_{f}'], bins=100, alpha=0.3, label=f'{f}-r')
plt.legend()
plt.title('LSST SMASSII synthetic colors')

plt.figure(figsize=(8,5))
for f in 'giz':
    plt.hist(df.loc[f'sdss_{f}'], bins=100, alpha=0.3, label=f'{f}-r')
plt.legend()
plt.title('SDSS SMASSII synthetic colors')

The bimodal distributions in g-r and r-i can be helpful in distinguishing very rough taxonomic classifications. These classifications are correlated with albedo distributions, and can be helpful in constraining size estimates.
See Ivezic & Ivezic 2021 https://www.sciencedirect.com/science/article/abs/pii/S0019103520305832 for an example.

In [ ]:
# bonus points - get the taxonomic classifications associated with the SMASS objects 
# and color-code the points in the color-color plots with this information 